# Watersheds Segmentation <a href="https://mybinder.org/v2/gh/InsightSoftwareConsortium/SimpleITK-Notebooks/master?filepath=Python%2F32_Watersheds_Segmentation.ipynb"><img style="float: right;" src="https://mybinder.org/badge_logo.svg"></a>

In [52]:
%matplotlib inline
import matplotlib.pyplot as plt
import SimpleITK as sitk
from myshow import myshow, myshow3d
import numpy as np

# Download data to work on
%run update_path_to_download_script
from downloaddata import fetch_data as fdata

import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import random

In [184]:
from os import walk

f = []
for (dirpath, dirnames, filenames) in walk('./data'):
    f.extend(dirnames)
    break
print(f)

['604.000000-COR 3X3-11320', '3.000000-0.625mm bone alg-26970', '2.000000-ROUTINE CHEST NON-CON-97100', '602.000000-SAG 3X3-00291', '605.000000-SAG 3X3-10651', '601.000000-COR 3X3-86740', '1.000000-SCOUT CHEST-97846']


In [221]:
reader = sitk.ImageSeriesReader()

def get_image(name):
    dicom_names = reader.GetGDCMSeriesFileNames(name)
    reader.SetFileNames(dicom_names)
    image = reader.Execute()
#     myshow(image) 
    return sitk.GetArrayFromImage(image)

def get_training_data(arr):
    print(arr)
    training_data = []
    first = True
    for fn in arr:
        path = str('./data/' + fn)
        print(path)
        image = get_image(path)
        print(len(image))
        if len(image) > 50:
            if first:
                training_data = [random.sample(list(image), 50)]
                first = False
            else:
                training_data = np.concatenate((training_data, [random.sample(list(image), 50)]))
        
    return training_data

training_data = get_training_data(f)




['604.000000-COR 3X3-11320', '3.000000-0.625mm bone alg-26970', '2.000000-ROUTINE CHEST NON-CON-97100', '602.000000-SAG 3X3-00291', '605.000000-SAG 3X3-10651', '601.000000-COR 3X3-86740', '1.000000-SCOUT CHEST-97846']
./data/604.000000-COR 3X3-11320


ImageSeriesReader (0x16d99d120): Non uniform sampling or missing slices detected,  maximum nonuniformity:176.649



111
(1, 50, 512, 512)
./data/3.000000-0.625mm bone alg-26970
510
(2, 50, 512, 512)
./data/2.000000-ROUTINE CHEST NON-CON-97100
255
(3, 50, 512, 512)
./data/602.000000-SAG 3X3-00291


ImageSeriesReader (0x108fd9050): Non uniform sampling or missing slices detected,  maximum nonuniformity:176.686



121
(4, 50, 512, 512)
./data/605.000000-SAG 3X3-10651


ImageSeriesReader (0x16d99d120): Non uniform sampling or missing slices detected,  maximum nonuniformity:176.682



121
(5, 50, 512, 512)
./data/601.000000-COR 3X3-86740
106
(6, 50, 512, 512)
./data/1.000000-SCOUT CHEST-97846
2


ImageSeriesReader (0x16d99d120): Non uniform sampling or missing slices detected,  maximum nonuniformity:176.653



In [228]:
training_data = np.concatenate((training_data, [random.sample(list(get_image('./healthy_lung')), 50)]))
print(np.shape(training_data))
print(training_data)

(7, 50, 512, 512)
[[[[-1024 -1024 -1024 ... -1024 -1024 -1024]
   [-1024 -1024 -1024 ... -1024 -1024 -1024]
   [-1024 -1024 -1024 ... -1024 -1024 -1024]
   ...
   [-1024 -1024 -1024 ... -1024 -1024 -1024]
   [-1024 -1024 -1024 ... -1024 -1024 -1024]
   [-1024 -1024 -1024 ... -1024 -1024 -1024]]

  [[-1024 -1024 -1024 ... -1024 -1024 -1024]
   [-1024 -1024 -1024 ... -1024 -1024 -1024]
   [-1024 -1024 -1024 ... -1024 -1024 -1024]
   ...
   [-1024 -1024 -1024 ... -1024 -1024 -1024]
   [-1024 -1024 -1024 ... -1024 -1024 -1024]
   [-1024 -1024 -1024 ... -1024 -1024 -1024]]

  [[-1024 -1024 -1024 ... -1024 -1024 -1024]
   [-1024 -1024 -1024 ... -1024 -1024 -1024]
   [-1024 -1024 -1024 ... -1024 -1024 -1024]
   ...
   [-1024 -1024 -1024 ... -1024 -1024 -1024]
   [-1024 -1024 -1024 ... -1024 -1024 -1024]
   [-1024 -1024 -1024 ... -1024 -1024 -1024]]

  ...

  [[-1024 -1024 -1024 ... -1024 -1024 -1024]
   [-1024 -1024 -1024 ... -1024 -1024 -1024]
   [-1024 -1024 -1024 ... -1024 -1024 -1024]
   

In [241]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(50, 512, 512)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64))
model.add(layers.Dense(32))
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_49 (Conv2D)          (None, 48, 510, 64)       294976    
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 24, 255, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_50 (Conv2D)          (None, 22, 253, 128)      73856     
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 11, 126, 128)     0         
 g2D)                                                            
                                                                 
 conv2d_51 (Conv2D)          (None, 9, 124, 128)       147584    
                                                                 
 flatten_14 (Flatten)        (None, 142848)          

In [242]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=3)

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['binary_accuracy'])


In [243]:
labels = np.array([1, 1,1,1,1,1, 0])
history = model.fit(training_data/255, labels, epochs=10, validation_data=(training_data/255, labels), callbacks=callback)

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 6.8288 - binary_accuracy: 0.5223 - val_loss: 5.6294 - val_binary_accuracy: 0.6339
Epoch 2/10
1/1 [==============================] - 2s 2s/step - loss: 5.6294 - binary_accuracy: 0.6339 - val_loss: 5.6294 - val_binary_accuracy: 0.6339
Epoch 3/10
1/1 [==============================] - 1s 1s/step - loss: 5.6294 - binary_accuracy: 0.6339 - val_loss: 5.6294 - val_binary_accuracy: 0.6339
Epoch 4/10
1/1 [==============================] - 1s 1s/step - loss: 5.6294 - binary_accuracy: 0.6339 - val_loss: 5.6294 - val_binary_accuracy: 0.6339


In [34]:

# list
# print(data['labelGroups'][2])
for i in range(0, len(data['datasets'])):
    for j in range(0, len(data['datasets'][i]['annotations'])):
        if '1.2.826.0.1.3680043.10.474.419639.312580455409613733097488204614' == data['datasets'][i]['annotations'][j].get('StudyInstanceUID', 0):
            print(data['datasets'][i]['annotations'][j]['labelId'])
    

L_ngve9m
L_ngve9m
L_MAYK1M
L_MAYK1M
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_MAYK1M
L_MAYK1M
L_ngve9m
L_MAYK1M
L_MAYK1M
L_MAYK1M
L_ngve9m
L_ngve9m
L_ngve9m
L_mvd9bJ
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L

In [4]:
import json
# Opening JSON file
f = open('MIDRC-RICORD-1a_annotations_labelgroup_all_2020-Dec-8 2.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)
 
# Iterating through the json
# list
for i in data:
    print(i)
 
# Closing file
f.close()

id
createdAt
updatedAt
name
description
isPrivate
labelGroups
datasets


In [24]:
data['datasets'][0]['annotations'][0]['SeriesInstanceUID']

'1.2.826.0.1.3680043.10.474.3116300553210485620889426707485524974'

In [3]:
feature_img = sitk.GradientMagnitude(image)
myshow(feature_img)

interactive(children=(IntSlider(value=127, description='z', max=254), Output()), _dom_classes=('widget-interac…

In [4]:
ws_img = sitk.MorphologicalWatershed(feature_img, level=0, markWatershedLine=True, fullyConnected=False)
myshow(sitk.LabelToRGB(ws_img), "Watershed Over Segmentation")

interactive(children=(IntSlider(value=127, description='z', max=254), Output()), _dom_classes=('widget-interac…

In [5]:
ws_img = sitk.MorphologicalWatershed(feature_img, level=50, markWatershedLine=True, fullyConnected=False)
myshow(sitk.LabelToRGB(ws_img), "Watershed Over Segmentation")

interactive(children=(IntSlider(value=127, description='z', max=254), Output()), _dom_classes=('widget-interac…